In [ ]:
from requests import get
from datetime import datetime
import time

Get new token
https://oauth.vk.com/authorize?client_id=6738602&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52

In [ ]:
token = '...'

In [ ]:
group_num = '...'
gender = ... #1 - female, 2 - male
offset_default = ... #0
today = '2019-08-06 00:00'
group_range = ...

In [ ]:
def get_city_title_for_id(id):
    if id == 1:
        return 'Moscow'
    else: return 'Not'

In [ ]:
class Person:
    def __init__(self, id, first_name, last_name, can_write_private_message):
        self.id = id
        self.first_name = first_name
        self.last_name = last_name
        self.can_write_private_message = can_write_private_message
        #self.is_closed = is_closed
        
        self.photo = ''

        self.sex = 0

        self.can_send_friend_request = 0

        self.friends_count = 0
        self.followers_count = 0

        self.city_id = 0
        self.computed_city_id = 0

        self.year = 0
        self.computed_year = 0

        #self.last_seen = last_seen    
        
    def printt(self):
        print("Person: {} {}".format(self.first_name, self.last_name))
        print("  sex:  {}".format("male" if self.sex == 2 else "female" if self.sex == 1 else "??"))
        print('  city: {}'.format(get_city_title_for_id(self.city_id)))
        print("  year: {}".format(self.year if self.year != 0 else ''))
        print("  can_write: {}".format("yes" if self.can_write_private_message == 1 else 'no'))
        print('  friends_count: {}'.format(str(self.friends_count) if self.friends_count != 0 else 'NaN'))
        print('  followers_count: {}'.format(str(self.followers_count) if self.followers_count != 0 else ''))
        print('  comp_year: {}'.format(str(self.computed_year) if self.computed_year != 0 else ''))
        print('  comp_city: {}'.format(get_city_title_for_id(self.computed_city_id)))
        print('  photo: {}'.format(self.photo))
        print('  city_id:  {}'.format(self.city_id))
        print('  id:  {}'.format(str(self.id)))

In [ ]:
class VKAPIError(Exception):
    pass
class VKAPIErrorTimeLimit(Exception):
    pass

In [ ]:
class API:
    def __init__(self, token=None, version=None):
        self.current_line = ''
        self.api_version = version or '5.9'
        if token is None:
            raise Exception('No token provided')
        self.access_token = token

    def __call__(self, *keys, **kwargs):
        payload = {
            'access_token': self.access_token,
            'v': self.api_version,
        }
        for key in kwargs:
            payload[key] = kwargs.get(key)

        r = get('https://api.vk.com/method/' + self.current_line, params=payload)

        self.current_line = ''

        if r.status_code != 200:
            print('something wrong with the connection')
        r = r.json()

        if 'response' in r:
            return r.get('response')
        if 'error' in r:
            error = r.get('error')
            if error.get('error_code', '') == 6:
                raise VKAPIErrorTimeLimit(error.get('error_msg'), error.get('request_params'))
            raise VKAPIError(r.get('error'))

        raise Exception('undefined behaviour')

    def __getattr__(self, key):
        if self.current_line == '':
            self.current_line = key
        else:
            self.current_line += '.' + key
        return self

In [ ]:
def get_year_from_bdate(bdate):
    items = bdate.strip().split('.')
    if len(items[-1]) == 4:
        return items[-1]
    else:
        return ''

In [ ]:
def get_person_from_item(item):
    first_name = item['first_name']
    last_name = item['last_name']
    id = item['id']
    can_write_private_message = item['can_write_private_message']
    #is_closed = item['is_closed']
    
    person = Person(id, first_name, last_name, can_write_private_message)

    person.sex = 0 if 'sex' not in item else item['sex']
    person.year = '' if 'bdate' not in item else get_year_from_bdate(item['bdate'])
    person.photo = '' if 'photo_max' not in item else item['photo_max']
    person.city_id = -1 if 'city' not in item else item['city']['id']
    person.last_seen = -1 if 'last_seen' not in item else datetime.utcfromtimestamp(item['last_seen']['time']).strftime('%Y-%m-%d %H:%M')
    return person

In [ ]:
def generate_html_people(items, result_file_name):
    template_text = open('py_crawler/template.html', 'r').read()
    output_file = open(result_file_name, 'w')

    tag_start = '$item_start$'
    tag_end = '$item_end$'

    container_start = template_text.find(tag_start)
    container_end = template_text.find(tag_end) + len(tag_end)


    output_file.write(template_text[:container_start])

    person_template = template_text[container_start + len(tag_start):container_end - len(tag_end)]
    for item in items:
        tmp = person_template.replace('$link$', item.get('person_link'))
        tmp = tmp.replace('$image$', item.get('image_link'))
        output_file.write(tmp)

    output_file.write(template_text[container_end:])

    output_file.close()

In [ ]:
import traceback
import time

def get_good_item(good_people):
    items = []
    i = 0
    k = 0
    for person in good_people:
        time.sleep(2)
        i = i + 1
        try:
            # Do something.
            t = api.friends.get(user_id=person.id, count = 500)
            tt= api.users.get(user_ids = person.id, fields = ['followers_count'])
            #print(len(t))
            #person.printt()
            #if len(t) > 50 and len(t) < 150:
            k = k + 1
            person.friends_count = t['count']
            person.followers_count = tt[0]['followers_count']
            items.append(person)
            pass
        except:
            # Continue to next iteration.
            #print(error_code)
            traceback.print_exc()
            continue
        if (k%100 == 0 and k != 0):
            print('yeeeah', k, i)
    return items

In [ ]:
if __name__ == '__main__':
    api = API(token = token)
    c = 0
    from time import time
    start_time = time()
    for i in range(2):
        while True:
            try:
                print(i, api.users.get(user_ids=['93603025'], 
                                       fields = ['sex, bdate, city, country, followers_count, can_write_private_message, last_seen']))
                c = c + 1
                print(c / (time()-start_time))
                break
            except VKAPIErrorTimeLimit:
                pass

In [ ]:
data = api.groups.getMembers(
                           group_id = group_num, sort = 'id_desc',
                           fields = ['sex, bdate, city, country, photo_max, can_write_private_message, last_seen'], 
                           count = 1000,
                           offset = offset_default
                          )

In [ ]:
for i in range(1, group_range):
    data2 = api.groups.getMembers(
                               group_id = group_num, sort = 'id_desc',
                               fields = ['sex, bdate, city, country, photo_max, can_write_private_message, last_seen'], 
                               count = 1000,
                               offset = offset_default + i * 1000
                              )
    data['items'].extend(data2['items'])

In [ ]:
good_people = [get_person_from_item(x) for x in data['items']]

In [ ]:
len(good_people)

In [ ]:
with open("py_crawler/test_arr.txt", "r") as file:
    arr = eval(file.readline())

In [ ]:
good_people = [person for person in good_people if person.id not in arr]
len(good_people)

In [ ]:
for pers in good_people:
    arr.append(pers.id)

In [ ]:
with open("py_crawler/test_arr.txt", "w") as file:
    file.write(str(arr))

In [ ]:
len(good_people)

In [ ]:
good_people[0].printt()

In [ ]:
len(good_people)

In [ ]:
very_good_people = get_good_item(good_people)

In [ ]:
very_good_people[165].printt()

In [ ]:
len(very_good_people)

In [ ]:
items = []
#female = 1
for person in very_good_people:
            if ( 
                
                (person.friends_count >= 500)  and (person.followers_count < 2350)
                 
               ):
                items.append({
                    'image_link' : person.photo,
                    'person_link' : 'https://vk.com/id' + str(person.id)
                    })

In [ ]:
len(items)

In [ ]:
generate_html_people(items, 'test.html')